# Start

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/My Drive/Biometric Analysis/data/train.csv', index_col = 'id')
test = pd.read_csv('/content/drive/My Drive/Biometric Analysis/data/test.csv', index_col = 'id')
submission = pd.read_csv('/content/drive/My Drive/Biometric Analysis/data/sample_submission.csv', index_col = 'id')

In [ ]:
SRC_COLUMNS = ['650_src', '660_src', '670_src', '680_src', '690_src',
       '700_src', '710_src', '720_src', '730_src', '740_src', '750_src',
       '760_src', '770_src', '780_src', '790_src', '800_src', '810_src',
       '820_src', '830_src', '840_src', '850_src', '860_src', '870_src',
       '880_src', '890_src', '900_src', '910_src', '920_src', '930_src',
       '940_src', '950_src', '960_src', '970_src', '980_src', '990_src']
DST_COLUMNS = ['650_dst', '660_dst', '670_dst', '680_dst', '690_dst', '700_dst',
       '710_dst', '720_dst', '730_dst', '740_dst', '750_dst', '760_dst',
       '770_dst', '780_dst', '790_dst', '800_dst', '810_dst', '820_dst',
       '830_dst', '840_dst', '850_dst', '860_dst', '870_dst', '880_dst',
       '890_dst', '900_dst', '910_dst', '920_dst', '930_dst', '940_dst',
       '950_dst', '960_dst', '970_dst', '980_dst', '990_dst']
TGT_COLUMNS = ['hhb', 'hbo2', 'ca', 'na']

In [ ]:
def interpolate_arr(xs):
    x = pd.DataFrame({'x': xs})
    return x['x'].interpolate().values

for df in [train]:
    df[DST_COLUMNS] = df.apply(lambda x: interpolate_arr(x[DST_COLUMNS].values), axis=1, result_type='expand')


In [ ]:
ZERO_COLUMNS = []
#absorbed lengths
for i,_ in enumerate(DST_COLUMNS):
    for df in [train, test]:
        df['zero'+DST_COLUMNS[i][:3]] = df[DST_COLUMNS[i]].isnull().astype(int)
    ZERO_COLUMNS.append('zero'+DST_COLUMNS[i][:3])

ZERO_SRC_COLUMNS = []
#absorbed lengths
for i,_ in enumerate(DST_COLUMNS):
    for df in [train, test]:
        df['zerosrc'+SRC_COLUMNS[i][:3]] = (df[SRC_COLUMNS[i]] == 0).astype(float)
    ZERO_SRC_COLUMNS.append('zerosrc'+DST_COLUMNS[i][:3])

LEN_COLUMNS = []
for i,_ in enumerate(SRC_COLUMNS):
    for df in [train, test]:
        df['len'+SRC_COLUMNS[i][:3]] = float(SRC_COLUMNS[i][:3])
    LEN_COLUMNS.append('len'+SRC_COLUMNS[i][:3])

def interpolate_arr(xs):
    x = pd.DataFrame({'x': xs})
    return x['x'].interpolate().values

for df in [train,test]:
    df[DST_COLUMNS] = df.apply(lambda x: interpolate_arr(x[DST_COLUMNS].values), axis=1, result_type='expand')

for i,_ in enumerate(DST_COLUMNS):
    for df in [train, test]:
        df[SRC_COLUMNS[i]] = np.log(df[SRC_COLUMNS[i]] + 1e-23)
        df[DST_COLUMNS[i]] = np.log(df[DST_COLUMNS[i]] + 1e-23)

In [ ]:
train

,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,730_src,740_src,750_src,760_src,770_src,780_src,790_src,800_src,810_src,820_src,830_src,840_src,850_src,860_src,870_src,880_src,890_src,900_src,910_src,920_src,930_src,940_src,950_src,960_src,970_src,980_src,990_src,650_dst,660_dst,670_dst,680_dst,...,zerosrc950,zerosrc960,zerosrc970,zerosrc980,zerosrc990,len650,len660,len670,len680,len690,len700,len710,len720,len730,len740,len750,len760,len770,len780,len790,len800,len810,len820,len830,len840,len850,len860,len870,len880,len890,len900,len910,len920,len930,len940,len950,len960,len970,len980,len990
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,25,-0.968901,-0.844133,-0.652466,-0.559211,-0.388343,-0.281355,-0.179366,-0.065894,-0.037359,0.015292,-0.001351,-0.057915,-0.168265,-0.300794,-0.374315,-0.549965,-0.662890,-0.832225,-1.007831,-1.051681,-1.202707,-1.309630,-1.461363,-1.514219,-1.613446,-1.931504,-2.028748,-2.084092,-2.226921,-2.121598,-2.317597,-2.210462,-2.600452,-2.554000,-2.893898,-40.551355,-40.807530,-52.959457,-39.588217,...,0.0,0.0,0.0,0.0,0.0,650.0,660.0,670.0,680.0,690.0,700.0,710.0,720.0,730.0,740.0,750.0,760.0,770.0,780.0,790.0,800.0,810.0,820.0,830.0,840.0,850.0,860.0,870.0,880.0,890.0,900.0,910.0,920.0,930.0,940.0,950.0,960.0,970.0,980.0,990.0
1,10,-52.959457,-52.959457,-4.010187,-52.959457,-52.959457,-3.925108,-5.741484,-52.959457,-52.959457,-4.346659,-52.959457,-4.157284,-52.959457,-52.959457,-4.354411,-4.483838,-5.629603,-52.959457,-52.959457,-52.959457,-52.959457,-52.959457,-3.108677,-2.231381,-1.530350,-0.880538,-0.450734,-0.171619,-0.025523,-0.001972,-0.028955,-0.138745,-0.283292,-0.455486,-0.621776,-52.959457,-52.959457,-52.959457,-22.334941,...,0.0,0.0,0.0,0.0,0.0,650.0,660.0,670.0,680.0,690.0,700.0,710.0,720.0,730.0,740.0,750.0,760.0,770.0,780.0,790.0,800.0,810.0,820.0,830.0,840.0,850.0,860.0,870.0,880.0,890.0,900.0,910.0,920.0,930.0,940.0,950.0,960.0,970.0,980.0,990.0
2,25,-52.959457,-3.414587,-3.723057,-3.321462,-2.839926,-2.406280,-1.900860,-1.683192,-1.363203,-1.189248,-0.942583,-0.683751,-0.563277,-0.389464,-0.226650,-0.132469,-0.074907,-0.005033,-0.013268,-0.010172,-0.050231,-0.106995,-0.225484,-0.381817,-0.543986,-0.771714,-0.938715,-1.277296,-1.657054,-1.795587,-2.152701,-2.696517,-3.753738,-3.237549,-3.462222,-43.231786,-44.726063,-45.418945,-52.959457,...,0.0,0.0,0.0,0.0,0.0,650.0,660.0,670.0,680.0,690.0,700.0,710.0,720.0,730.0,740.0,750.0,760.0,770.0,780.0,790.0,800.0,810.0,820.0,830.0,840.0,850.0,860.0,870.0,880.0,890.0,900.0,910.0,920.0,930.0,940.0,950.0,960.0,970.0,980.0,990.0
3,10,-1.290875,-1.162159,-1.111758,-0.890599,-0.763849,-0.639246,-0.440538,-0.307042,-0.224832,-0.089028,-0.048413,-0.006109,0.005614,-0.070798,-0.166019,-0.287722,-0.389124,-0.537307,-0.680389,-0.841949,-1.030468,-1.178110,-1.282944,-1.422170,-1.601569,-1.721711,-1.887896,-1.854316,-2.010513,-1.948624,-2.290064,-2.309811,-2.381737,-2.251607,-2.756873,-23.930961,-23.781902,-23.652207,-25.421990,...,0.0,0.0,0.0,0.0,0.0,650.0,660.0,670.0,680.0,690.0,700.0,710.0,720.0,730.0,740.0,750.0,760.0,770.0,780.0,790.0,800.0,810.0,820.0,830.0,840.0,850.0,860.0,870.0,880.0,890.0,900.0,910.0,920.0,930.0,940.0,950.0,960.0,970.0,980.0,990.0
4,15,0.015095,0.008682,-0.010758,-0.011324,0.017575,0.016188,0.000090,-0.017991,0.015519,0.002158,0.011830,0.003434,-0.016485,0.013410,0.016356,-0.017960,-0.009132,-0.007448,0.001748,-0.011455,-0.017451,0.004311,0.014238,0.014445,-0.011455,-0.017146,0.006141,-0.009596,0.013124,0.015470,0.010663,-0.005626,-0.014129,-0.001111,0.014021,-30.670757,-31.363905,-52.959457,-29.791161,...,0.0,0.0,0.0,0.0,0.0,650.0,660.0,670.0,680.0,690.0,700.0,710.0,720.0,730.0,740.0,750.0,760.0,770.0,780.0,790.0,800.0,810.0,820.0,830.0,840.0,850.0,860.0,870.0,880.0,890.0,900.0,910.0,920.0,930.0,940.0,950.0,960.0,970.0,980.0,990.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
dst_columns = [k for k in train.columns if 'dst' in k]
train_dst = train[dst_columns]
test_dst = test[dst_columns]
train[dst_columns] = train_dst.interpolate(axis=1)
train.fillna(0, inplace=True)
test[dst_columns] = test_dst.interpolate(axis=1)
test.fillna(0, inplace=True)

In [ ]:
x_train = train.loc[:, 'rho':'990_dst']
y_train = train.loc[:, 'hhb':'na']
x_test = test.loc[:,'rho':'990_dst']

In [ ]:
src_list = []
dst_list = []
for i in range(650, 1000, 10):
  dst_list.append('{}_dst'.format(i))
  src_list.append('{}_src'.format(i))

# Function

In [ ]:
from sklearn.decomposition import PCA, FastICA, KernelPCA

In [ ]:
def PCATrans(df_train, df_test, col):
  global x_train
  global x_test
  pca = PCA()
  pca.fit(df_train)
  cumsum = np.cumsum(pca.explained_variance_ratio_)
  num_d = np.argmax(cumsum >= 0.98) + 1
  print(num_d)
  col_list = []
  for i in range(1, num_d+1):
    col_list.append(col+'_pca_{}'.format(i))
  pca = PCA(n_components = num_d)
  new_d = pca.fit_transform(df_train)
  new_dd = pca.fit_transform(df_test)
  x_train = pd.concat([x_train, pd.DataFrame(new_d, columns = col_list)], axis=1)
  x_test = pd.concat([x_test, pd.DataFrame(new_dd, columns = col_list, index = x_test.index)],axis=1)

In [ ]:
def FastICATrans(df_train, df_test, col):
  global x_train
  global x_test
  ICA = FastICA(n_components=4, random_state=0)
  col_list = []
  for i in range(1,5):
    col_list.append(col+'_ica_{}'.format(i))
  new_d = ICA.fit_transform(df_train)
  new_dd = ICA.fit_transform(df_test)
  x_train = pd.concat([x_train, pd.DataFrame(new_d, columns = col_list)], axis=1)
  x_test = pd.concat([x_test, pd.DataFrame(new_dd, columns = col_list, index = x_test.index)],axis=1)

In [ ]:
def KPCATrans(df_train, df_test, col):
  global x_train
  global x_test
  KPCA = KernelPCA(n_components=4, kernel='rbf')
  col_list = []
  for i in range(1,5):
    col_list.append(col+'_kpca_{}'.format(i))
  new_d = KPCA.fit_transform(df_train)
  new_dd = KPCA.fit_transform(df_test)
  x_train = pd.concat([x_train, pd.DataFrame(new_d, columns = col_list)], axis=1)
  x_test = pd.concat([x_test, pd.DataFrame(new_dd, columns = col_list, index = x_test.index)],axis=1)

# Features Extract

dst PCA+ICA+KPCA

In [ ]:
PCATrans(x_train[dst_list], x_test[dst_list], 'dst')
FastICATrans(x_train[dst_list], x_test[dst_list], 'dst')
KPCATrans(x_train[dst_list], x_test[dst_list], 'dst')

5


Transmittance

In [ ]:
rate_list = []
epsilon = 1.0e-30
for i in tqdm(range(650, 1000, 10)):
  rate_list.append('{}_src_dst_rate'.format(i))
  x_train['{}_src_dst_rate'.format(i)] = x_train.loc[:,'{}_dst'.format(i)]/(x_train.loc[:,'{}_src'.format(i)]+epsilon)
  for idx, k in enumerate(x_train['{}_src_dst_rate'.format(i)]):
    if k>1:
      x_train.loc[idx, '{}_src_dst_rate'.format(i)] = 0
  x_test['{}_src_dst_rate'.format(i)] = x_test.loc[:,'{}_dst'.format(i)]/(x_test.loc[:,'{}_src'.format(i)]+epsilon)
  for idx, k in enumerate(x_test['{}_src_dst_rate'.format(i)]):
    if k>1:
      x_test.loc[idx+10000, '{}_src_dst_rate'.format(i)] = 0

100%|██████████| 35/35 [00:13<00:00,  2.65it/s]


In [ ]:
PCATrans(x_train[rate_list], x_test[rate_list], 'rate')
FastICATrans(x_train[rate_list], x_test[rate_list], 'rate')
KPCATrans(x_train[rate_list], x_test[rate_list], 'rate')

17


Absorbance

In [ ]:
absorbance_list = []
epsilon = 1.0e-30
for i in tqdm(range(650, 1000, 10)):
  absorbance_list.append('{}_src_dst_absor'.format(i))
  x_train['{}_src_dst_absor'.format(i)] = np.log10((x_train['{}_src'.format(i)]+epsilon)/(x_train['{}_dst'.format(i)]+epsilon)).clip(0)
  x_test['{}_src_dst_absor'.format(i)] = np.log10((x_test['{}_src'.format(i)]+epsilon)/(x_test['{}_dst'.format(i)]+epsilon)).clip(0)

100%|██████████| 35/35 [00:00<00:00, 204.73it/s]


In [ ]:
PCATrans(x_train[absorbance_list], x_test[absorbance_list], 'absorbance')
FastICATrans(x_train[absorbance_list], x_test[absorbance_list], 'absorbance')
KPCATrans(x_train[absorbance_list], x_test[absorbance_list], 'absorbance')

30


Absorption Coefficient

In [ ]:
Absorcoef_list = []
epsilon = 1.0e-30
for i in tqdm(range(650, 1000, 10)):
  Absorcoef_list.append('{}_src_dst_absor_coef'.format(i))
  x_train['{}_src_dst_absor_coef'.format(i)] = (np.log((x_train['{}_src'.format(i)]+epsilon)/(x_train['{}_dst'.format(i)]+epsilon)).clip(0))/x_train['rho']
  x_test['{}_src_dst_absor_coef'.format(i)] = (np.log((x_test['{}_src'.format(i)]+epsilon)/(x_test['{}_dst'.format(i)]+epsilon)).clip(0))/x_test['rho']

100%|██████████| 35/35 [00:00<00:00, 186.35it/s]


In [ ]:
PCATrans(x_train[Absorcoef_list], x_test[Absorcoef_list], 'Absorcoef')
FastICATrans(x_train[Absorcoef_list], x_test[Absorcoef_list], 'Absorcoef')
KPCATrans(x_train[Absorcoef_list], x_test[Absorcoef_list], 'Absorcoef')

30


Daubechies Wavelet Transform(db4, level = 4)

In [ ]:
import pywt
from scipy.signal import hilbert

In [ ]:
train_coeffs = pywt.wavedec(x_train.loc[:,dst_list], 'db4', level = 4)
test_coeffs = pywt.wavedec(x_test.loc[:,dst_list], 'db4', level = 4)
train_cA4, train_cD4, train_cD3, train_cD2, train_cD1 = train_coeffs
test_cA4, test_cD4, test_cD3, test_cD2, test_cD1 = test_coeffs
cA4_list = []
cD4_list = []
cD3_list = []
cD2_list = []
cD1_list = []
for i in range(1,25):
  cA4_list.append('cA4_hil_{}'.format(i))
  cD4_list.append('cD4_hil_{}'.format(i))
for i in range(1,31):
  cD3_list.append('cD3_hil_{}'.format(i))
for i in range(1,43):
  cD2_list.append('cD2_hil_{}'.format(i))
for i in range(1,64):
  cD1_list.append('cD1_hil_{}'.format(i))
analytic_signal = hilbert(train_cA4)
amplitude_envelope = np.abs(analytic_signal)
train_cA4_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cA4_list)
analytic_signal = hilbert(train_cD4)
amplitude_envelope = np.abs(analytic_signal)
train_cD4_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD4_list)
analytic_signal = hilbert(train_cD3)
amplitude_envelope = np.abs(analytic_signal)
train_cD3_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD3_list)
analytic_signal = hilbert(train_cD2)
amplitude_envelope = np.abs(analytic_signal)
train_cD2_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD2_list)
analytic_signal = hilbert(train_cD1)
amplitude_envelope = np.abs(analytic_signal)
train_cD1_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD1_list)
x_train = pd.concat([x_train, train_cA4_df, train_cD4_df, train_cD3_df, train_cD2_df, train_cD1_df], axis=1)

analytic_signal = hilbert(test_cA4)
amplitude_envelope = np.abs(analytic_signal)
test_cA4_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cA4_list, index = x_test.index)
analytic_signal = hilbert(test_cD4)
amplitude_envelope = np.abs(analytic_signal)
test_cD4_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD4_list, index = x_test.index)
analytic_signal = hilbert(test_cD3)
amplitude_envelope = np.abs(analytic_signal)
test_cD3_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD3_list, index = x_test.index)
analytic_signal = hilbert(test_cD2)
amplitude_envelope = np.abs(analytic_signal)
test_cD2_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD2_list, index = x_test.index)
analytic_signal = hilbert(test_cD1)
amplitude_envelope = np.abs(analytic_signal)
test_cD1_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD1_list, index = x_test.index)
x_test = pd.concat([x_test, test_cA4_df, test_cD4_df, test_cD3_df, test_cD2_df, test_cD1_df], axis=1)

In [ ]:
PCATrans(x_train[cA4_list+cD4_list+cD3_list+cD2_list+cD1_list], x_test[cA4_list+cD4_list+cD3_list+cD2_list+cD1_list], 'wavelet')
FastICATrans(x_train[cA4_list+cD4_list+cD3_list+cD2_list+cD1_list], x_test[cA4_list+cD4_list+cD3_list+cD2_list+cD1_list], 'wavelet')
KPCATrans(x_train[cA4_list+cD4_list+cD3_list+cD2_list+cD1_list], x_test[cA4_list+cD4_list+cD3_list+cD2_list+cD1_list], 'wavelet')

5




---



번외 wavelet hilbert(level=3)

In [ ]:
train_coeffs = pywt.wavedec(x_train.loc[:,dst_list], 'db4', level = 3)
test_coeffs = pywt.wavedec(x_test.loc[:,dst_list], 'db4', level = 3)
train_cA3, train_cD3, train_cD2, train_cD1 = train_coeffs
test_cA3, test_cD3, test_cD2, test_cD1 = test_coeffs
cA3_list = []
cD3_list = []
cD2_list = []
cD1_list = []
for i in range(1,31):
  cA3_list.append('cA3_hil_{}'.format(i))
  cD3_list.append('cD3_hil_{}'.format(i))
for i in range(1,43):
  cD2_list.append('cD2_hil_{}'.format(i))
for i in range(1,64):
  cD1_list.append('cD1_hil_{}'.format(i))
analytic_signal = hilbert(train_cA3)
amplitude_envelope = np.abs(analytic_signal)
train_cA3_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cA3_list)
analytic_signal = hilbert(train_cD3)
amplitude_envelope = np.abs(analytic_signal)
train_cD3_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD3_list)
analytic_signal = hilbert(train_cD2)
amplitude_envelope = np.abs(analytic_signal)
train_cD2_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD2_list)
analytic_signal = hilbert(train_cD1)
amplitude_envelope = np.abs(analytic_signal)
train_cD1_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD1_list)
x_train = pd.concat([x_train, train_cA3_df, train_cD3_df, train_cD2_df, train_cD1_df], axis=1)

analytic_signal = hilbert(test_cA3)
amplitude_envelope = np.abs(analytic_signal)
test_cA3_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cA3_list, index = x_test.index)
analytic_signal = hilbert(test_cD3)
amplitude_envelope = np.abs(analytic_signal)
test_cD3_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD3_list, index = x_test.index)
analytic_signal = hilbert(test_cD2)
amplitude_envelope = np.abs(analytic_signal)
test_cD2_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD2_list, index = x_test.index)
analytic_signal = hilbert(test_cD1)
amplitude_envelope = np.abs(analytic_signal)
test_cD1_df = pd.DataFrame(np.concatenate((analytic_signal.real, analytic_signal.imag, amplitude_envelope), axis=1), columns = cD1_list, index = x_test.index)
x_test = pd.concat([x_test, test_cA3_df, test_cD3_df, test_cD2_df, test_cD1_df], axis=1)

/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))


Fourier Transform (FFT)

In [ ]:
from scipy.fftpack import fft

In [ ]:
x_train_dst = x_train[dst_list]
for i in tqdm(x_train_dst.index):
  x_train_dst.loc[i] = x_train_dst.loc[i] - x_train_dst.loc[i].mean()
  x_train.loc[i,dst_list] = x_train_dst.loc[i]
x_test_dst = x_test[dst_list]
for i in tqdm(x_test_dst.index):
  x_test_dst.loc[i] = x_test_dst.loc[i] - x_test_dst.loc[i].mean()
  x_test.loc[i, dst_list] = x_test_dst.loc[i]
x_train_fft = fft(x_train[dst_list])
x_test_fft = fft(x_test[dst_list])
for i in range(1,36):
  x_train['{}_real_fft'.format(i)] = np.real(x_train_fft)[:,i-1]
  x_test['{}_real_fft'.format(i)] = np.real(x_test_fft)[:,i-1]
  x_train['{}_imag_fft'.format(i)] = np.imag(x_train_fft)[:,i-1]
  x_test['{}_imag_fft'.format(i)] = np.imag(x_test_fft)[:,i-1]

  0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
  0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

In [ ]:
fft_list = []
for i in range(1, 36):
  fft_list.append('{}_real_fft'.format(i))
  fft_list.append('{}_imag_fft'.format(i))

In [ ]:
PCATrans(x_train[fft_list], x_test[fft_list], 'fft')
FastICATrans(x_train[fft_list], x_test[fft_list], 'fft')
KPCATrans(x_train[fft_list], x_test[fft_list], 'fft')

6


Power Spectrum Density(FFT)

In [ ]:
from scipy import signal

In [ ]:
freqs, psd_train = signal.welch(x_train_fft)
freqs, psd_test = signal.welch(x_test_fft)
psd_list = []
for i in range(1,36):
  psd_list.append(str(i)+'_psd')
psd_train = pd.DataFrame(psd_train, columns = psd_list)
psd_test = pd.DataFrame(psd_test, columns = psd_list, index = x_test.index)
x_train = pd.concat([x_train, psd_train], axis=1)
x_test = pd.concat([x_test, psd_test], axis=1)

/usr/local/lib/python3.6/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 35, using nperseg = 35
  .format(nperseg, input_length))
/usr/local/lib/python3.6/dist-packages/scipy/signal/spectral.py:1815: UserWarning: Input data is complex, switching to return_onesided=False
  warnings.warn('Input data is complex, switching to '


DCT

In [ ]:
from scipy.fftpack import dct

In [ ]:
x_train_dct = dct(x_train_fft)
x_test_dct = dct(x_test_fft)
for i in range(1,36):
  x_train['{}_real_dct'.format(i)] = np.real(x_train_dct)[:,i-1]
  x_test['{}_real_dct'.format(i)] = np.real(x_test_dct)[:,i-1]
  x_train['{}_imag_dct'.format(i)] = np.imag(x_train_dct)[:,i-1]
  x_test['{}_imag_dct'.format(i)] = np.imag(x_test_dct)[:,i-1]

square rho

In [ ]:
x_train['rho'] = x_train['rho']**2
x_test['rho'] = x_test['rho']**2

In [ ]:
print(x_train.shape, x_test.shape)

(10000, 534) (10000, 534)


# 'ca' score monitoring

LGBMRegressor

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [ ]:
base_params={'n_estimators': 1000, 'num_leaves': 50, 'learning_rate': 0.1, 
             'colsample_bytree': 1, 'subsample': 1, 'max_depth' : -1, 
             'reg_alpha': 4, 'reg_lambda': 7, 'min_data_in_leaf': 25}
LGBM_model = LGBMRegressor(objective='regression', subsample_freq=1, silent=False, random_state=18, 
                         importance_type='gain', **base_params)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train['ca'], test_size=0.1, random_state=1)

eval_set = [(X_train, Y_train),(X_test, Y_test)]
model_ = LGBM_model.fit(X_train, Y_train, eval_metric = ['mae'], eval_set=eval_set, verbose=True)

[1]	training's l1: 2.31679	training's l2: 8.41534	valid_1's l1: 2.25499	valid_1's l2: 7.91086
[2]	training's l1: 2.24688	training's l2: 7.9305	valid_1's l1: 2.19351	valid_1's l2: 7.52038
[3]	training's l1: 2.1879	training's l2: 7.53302	valid_1's l1: 2.14698	valid_1's l2: 7.23242
[4]	training's l1: 2.13445	training's l2: 7.17811	valid_1's l1: 2.1052	valid_1's l2: 6.97138
[5]	training's l1: 2.08635	training's l2: 6.86672	valid_1's l1: 2.06943	valid_1's l2: 6.76696
[6]	training's l1: 2.04103	training's l2: 6.58218	valid_1's l1: 2.03914	valid_1's l2: 6.56229
[7]	training's l1: 1.99571	training's l2: 6.31284	valid_1's l1: 2.00451	valid_1's l2: 6.37492
[8]	training's l1: 1.95708	training's l2: 6.0841	valid_1's l1: 1.97837	valid_1's l2: 6.21271
[9]	training's l1: 1.92091	training's l2: 5.87635	valid_1's l1: 1.95182	valid_1's l2: 6.07432
[10]	training's l1: 1.8878	training's l2: 5.68434	valid_1's l1: 1.93261	valid_1's l2: 5.95296
[11]	training's l1: 1.85575	training's l2: 5.50547	valid_1's l1:

XGBRegressor

In [ ]:
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib
import sklearn
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
base_params={'n_estimators': 1000, 'learning_rate': 0.1, 
             'colsample_bytree': 1, 'subsample': 1, 'max_depth' : 5}

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', subsample_freq=1, silent=False, random_state=18, 
                         importance_type='gain', **base_params, n_jobs = -1)

X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train['ca'], test_size=0.1, random_state=1)

eval_set = [(X_train, Y_train),(X_test, Y_test)]
model_ = xgb_model.fit(X_train, Y_train, eval_metric = ['mae'], eval_set=eval_set, verbose=True)

[0]	validation_0-mae:7.66711	validation_1-mae:7.74052
[1]	validation_0-mae:6.90837	validation_1-mae:6.98528
[2]	validation_0-mae:6.22877	validation_1-mae:6.30466
[3]	validation_0-mae:5.62271	validation_1-mae:5.6936
[4]	validation_0-mae:5.08293	validation_1-mae:5.15566
[5]	validation_0-mae:4.60393	validation_1-mae:4.67272
[6]	validation_0-mae:4.18121	validation_1-mae:4.24886
[7]	validation_0-mae:3.81054	validation_1-mae:3.88197
[8]	validation_0-mae:3.48627	validation_1-mae:3.56187
[9]	validation_0-mae:3.20537	validation_1-mae:3.28071
[10]	validation_0-mae:2.96533	validation_1-mae:3.03995
[11]	validation_0-mae:2.75889	validation_1-mae:2.83712
[12]	validation_0-mae:2.5825	validation_1-mae:2.6666
[13]	validation_0-mae:2.43455	validation_1-mae:2.52092
[14]	validation_0-mae:2.30831	validation_1-mae:2.39594
[15]	validation_0-mae:2.20243	validation_1-mae:2.29167
[16]	validation_0-mae:2.11458	validation_1-mae:2.20384
[17]	validation_0-mae:2.03718	validation_1-mae:2.12816
[18]	validation_0-mae:1

# cv score monitoring

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.multioutput import MultiOutputRegressor
import warnings ; warnings.filterwarnings('ignore')
import time

In [ ]:
base_params={'n_estimators': 1000, 'num_leaves': 50, 'learning_rate': 0.1, 
             'colsample_bytree': 1, 'subsample': 1, 'max_depth' : -1, 
             'reg_alpha': 4, 'reg_lambda': 7, 'min_data_in_leaf': 25}
LGBM_model = LGBMRegressor(objective='regression', subsample_freq=1, silent=False, random_state=18, 
                         importance_type='gain', **base_params)
multi_model=MultiOutputRegressor(LGBM_model)

In [ ]:
def model_scoring_cv(model, x, y, cv=10):
    start=time.time()
    print("Start Time : ", time.strftime('%c', time.localtime(time.time())) )
    score=-cross_val_score(model, x, y, cv=cv, scoring='neg_mean_absolute_error').mean()
    stop=time.time()
    print(f"Validation Time : {round(stop-start, 3)} sec")
    return score

In [ ]:
model_scoring_cv(multi_model, x_train, y_train)

Start Time :  Sat Jun 20 19:08:49 2020
Validation Time : 3441.882 sec


0.9794677587026086

# Predict

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
base_params={'n_estimators': 1000, 'num_leaves': 50, 'learning_rate': 0.1, 
             'colsample_bytree': 1, 'subsample': 1, 'max_depth' : -1, 
             'reg_alpha': 4, 'reg_lambda': 7, 'min_data_in_leaf': 25}
LGBM_model = LGBMRegressor(objective='regression', subsample_freq=1, silent=False, random_state=18, 
                         importance_type='gain', **base_params)
multi_model=MultiOutputRegressor(LGBM_model)

In [ ]:
multi_model.fit(x_train, y_train)

MultiOutputRegressor(estimator=LGBMRegressor(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1,
                                             importance_type='gain',
                                             learning_rate=0.1, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_data_in_leaf=25,
                                             min_split_gain=0.0,
                                             n_estimators=1000, n_jobs=-1,
                                             num_leaves=50,
                                             objective='regression',
                                             random_state=18, reg_alpha=4,
                                             reg_lambda=7, silent=False,
                                      

In [ ]:
preds=multi_model.predict(x_test)

In [ ]:
preds=pd.DataFrame(data=preds, columns=submission.columns, index=submission.index)
preds.head()

,hhb,hbo2,ca,na
id,,,,
10000,8.298320,4.914414,8.232432,2.980229
10001,8.693218,2.073751,8.774958,2.630830
10002,9.286215,4.977984,12.187749,4.316741
10003,8.193589,4.035437,9.590046,4.367109
10004,7.566014,3.161260,7.911440,3.121992


In [ ]:
preds.to_csv('/content/drive/My Drive/Biometric Analysis/DaconOptical_LGBM_mul_fftPSD_fixparams_0_9794.csv')